In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.applications import VGG16
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
import matplotlib.pyplot as plt

In [ ]:
root_dir = '../dataset/mnist/'
X, y = [], []
for root, dirs, files in os.walk(root_dir):
    if len(files) == 0:
        continue
    
    letter_number = int(root.split('/')[-1])
    y_temp = [0] * 26
    # print(letter_number)
    y_temp[letter_number-1] = 1

    for f in tqdm(files):
        image_filepath = os.path.join(root, f)
        image = Image.open(image_filepath)
        image = ImageOps.grayscale(image)
        image = np.array(image)
        X.append(image)
        y.append(y_temp.copy())
    

In [ ]:
X = np.array(X).reshape(-1, 28, 28, 1) / 255.0
y = np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
with open('/content/drive/MyDrive/Datasets/license-plate-detection/X_letters.pkl', 'wb') as f:
    pickle.dump(X, f)
with open('/content/drive/MyDrive/Datasets/license-plate-detection/y_letters.pkl', 'wb') as f:
    pickle.dump(y, f)

In [ ]:
X = None
y = None

with open('/content/drive/MyDrive/Datasets/license-plate-detection/X_numbers.pkl', 'rb') as f:
    X = pickle.load(f)
with open('/content/drive/MyDrive/Datasets/license-plate-detection/y_numbers.pkl', 'rb') as f:
    y = pickle.load(f)

In [ ]:
index = 100
img = X[index].reshape(28, 28)
label_index = np.argmax(y[index])
plt.title(label_index)
plt.imshow(img, cmap='gray')

In [ ]:
model = Sequential(name='letter_classifier')

model.add(Conv2D( filters=16, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D( filters=64, kernel_size=(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D( filters=256, kernel_size=(3,3), activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=26, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=256, epochs=40, validation_split=0.2, shuffle=True)

In [ ]:
model.save("/content/drive/MyDrive/Datasets/license-plate-detection/letter_classifier")